In [3]:
import tensorflow as tf
#from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

메모리 용량에 맞지 않는 아주 큰 규모의 데이터셋으로 딥러닝 시스템을 훈련해야 하는 경우가 많다. 텐서플로우는 데이터 API를 통해 데이터셋 객체를 만들고 데이터를 읽어 올 위치와 변환 방법을 지정하면 된다. 텐서플로우가 멀티스레딩, 큐, 배치, 프리페치 같은 상세한 사항을 모두 대신 처리해준다.

# 데이터 API

데이터API의 중심에는 데이터셋이라는 개념이 있다. 이는 연속된 데이터 샘플을 나타낸다. 일반적으로 디스크에서 데이털르 점진적으로 읽는 데이터셋을 사용한다.

In [12]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

`from_tensor_slices()`함수는 텐서를 받아 X의 각 원소가 아이템으로 표현되는 *tf.data.Dataset* 을 만든다. 즉 이 데이터셋은 텐서 0, 1, 2, ..., 9에 해당하는 10개의 아이템을 가진다. 이 경우 *tf.data.Dataset.range(10)* 으로 만든 데이터셋과 동일하다.

In [13]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


## 연쇄 변환

데이터셋이 준비되면 변환 메서드를 호출하여 여러 종류의 변환을 수행할 수 있다. 각 메서드는 새로운 데이터셋을 반환하므로 다음과 같이 변환 메서드를 연결할 수 있다.

In [14]:
dataset2 = dataset.repeat(3).batch(7)
for item in dataset2:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


`map()` 메서드를 호출하여 아이템을 변환할 수도 있다.

In [16]:
dataset3 = dataset.map(lambda x: x*2)
for item in dataset3:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)


`map()` 함수는 데이터에 원하는 어떤 전처리 작업에도 적용할 수 있다. 이따금 이미지 크기 변환이나 회전 같은 복잡한 계산을 포함하기 때문에 여러 스레드로 나누어 속도를 높이는 것이 좋다. *num_parallel_calls* 매개변수를 지정하면 간단하다.

`map()` 메서드가 각 아이템에 변환을 적용하지만 `aply()` 메서드는 데이터셋 전체에 변환을 적용한다.

In [17]:
dataset4 = dataset2.apply(tf.data.experimental.unbatch())

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.


In [18]:
for item in dataset4:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype

In [19]:
for item in dataset4.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


## 데이터 셔플링

경사 하강법은 훈련 세트에 있는 샘플이 독립적이고 동일한 분포일 때 최고의 성능을 발휘한다. 이렇게 하는 간단한 방법은 `shuffle()` 메서드를 사용하여 샘플을 섞는 것이다. 이 메서드는 먼저 원본 데이터셋의 처음 아이템을 `buffer_size` 개수만큼 추출하여 버퍼에 채운다. 그다음 새로운 아이템이 요청되면 이 버퍼에서 랜덤하게 하나를 꺼내 반환한다. 그리고 원본 데이터셋에서 새로운 아이템을 추출하여 비워진 버퍼를 채운다. 원본 데이터셋의 모든 아이템이 사용될 때까지 반복된다. 이 메서드를 사용하려면 버퍼 크기를 지정해야 한다. 버퍼 크기를 충분히 크게 하는 것이 중요하다. 다만 보유한 메모리의 크기를 넘지 않아야 한다.

In [20]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


메모리 용량보다 큰 대규모 데이터셋은 버퍼가 데이터셋에 비해 작기 때문에 간단한 셔플링 버퍼 방식으로는 충분하지 않다. 이를 해결하는 방법은 원본 데이터 자체를 섞는 것이다. 원본 데이터가 섞여 있더라고 일반적으로 에포크마다 한번 더 섞어주지 않으면 에포크마다 동일한 순서가 반복되어 모델에 편향이 추가된다.

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)